In [2]:
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
from torchvision import datasets, transforms
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [3]:
warnings.filterwarnings(action="ignore")

class EarlyStopping:
    def __init__(self, patience=3, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_val_loss = float('inf')
        self.best_val_acc = 0.0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, val_accuracy, model):
        improved = False

        if val_loss < (self.best_val_loss - self.min_delta):
            self.best_val_loss = val_loss
            improved = True
        if val_accuracy > (self.best_val_acc + self.min_delta):
            self.best_val_acc = val_accuracy
            improved = True

        if improved:
            self.counter = 0
            self.best_model_state = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

def calculate_metrics(y_true, y_pred):
    """Calculate multiple classification metrics"""
    metrics = {
        'accuracy': np.mean(y_true == y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def calculate_class_metrics(y_true, y_pred, classes):
    """Calculate metrics per class"""
    results = {}
    for i, class_name in enumerate(classes):
        true_positives = np.sum((y_true == i) & (y_pred == i))
        support = np.sum(y_true == i)
        
        results[class_name] = {
            'true_positives': true_positives,
            'precision': precision_score(y_true == i, y_pred == i, zero_division=0),
            'recall': recall_score(y_true == i, y_pred == i, zero_division=0),
            'f1': f1_score(y_true == i, y_pred == i, zero_division=0),
            'support': support
        }
    return results

# Path to dataset and transformations
data_dir = 'Mastercard dataset'
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Load and split dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
indices = list(range(len(dataset)))
labels = [label for _, label in dataset]

# Stratified train/val/test split (60/20/20)
train_idx, temp_idx = [], []
for label in np.unique(labels):
    label_idx = np.where(np.array(labels) == label)[0]
    np.random.shuffle(label_idx)
    split = int(0.6 * len(label_idx))
    train_idx.extend(label_idx[:split])
    temp_idx.extend(label_idx[split:])

val_idx, test_idx = [], []
for label in np.unique(labels):
    label_idx = [i for i in temp_idx if labels[i] == label]
    np.random.shuffle(label_idx)
    split = int(0.5 * len(label_idx))
    val_idx.extend(label_idx[:split])
    test_idx.extend(label_idx[split:])

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, 
                         sampler=SubsetRandomSampler(train_idx), num_workers=2)
val_loader = DataLoader(dataset, batch_size=batch_size,
                       sampler=SubsetRandomSampler(val_idx), num_workers=2)
test_loader = DataLoader(dataset, batch_size=batch_size,
                        sampler=SubsetRandomSampler(test_idx), num_workers=2)

# Model definition
class LogoClassifier(nn.Module):
    def __init__(self):
        super(LogoClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 16 * 16)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LogoClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)
early_stopping = EarlyStopping(patience=5, min_delta=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    val_preds, val_labels = [], []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_preds.extend(predicted.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    scheduler.step()
    
    # Calculate metrics
    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    train_accuracy = correct / total
    val_accuracy = val_correct / val_total
    val_metrics = calculate_metrics(np.array(val_labels), np.array(val_preds))

    # Print epoch summary
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print(f"{'Training:':<12} Loss: {avg_train_loss:.4f} | Acc: {100*train_accuracy:.2f}%")
    print(f"{'Validation:':<12} Loss: {avg_val_loss:.4f} | Acc: {100*val_accuracy:.2f}%")
    print(f"{'':<12} Precision: {100*val_metrics['precision']:.2f}%")
    print(f"{'':<12} Recall: {100*val_metrics['recall']:.2f}%")
    print(f"{'':<12} F1-score: {100*val_metrics['f1']:.2f}%")

    # Early stopping check
    early_stopping(avg_val_loss, val_accuracy, model)
    if early_stopping.early_stop:
        print("\nEarly stopping triggered! Loading best model...")
        model.load_state_dict(early_stopping.best_model_state)
        break

# Final test evaluation
model.eval()
test_preds, test_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_metrics = calculate_metrics(np.array(test_labels), np.array(test_preds))
class_metrics = calculate_class_metrics(np.array(test_labels), np.array(test_preds), dataset.classes)

# Print test results
print("\n" + "="*60)
print("Final Test Set Performance:")
print("="*60)
print(f"{'Accuracy:':<15} {100*test_metrics['accuracy']:.2f}%")
print(f"{'Precision:':<15} {100*test_metrics['precision']:.2f}%")
print(f"{'Recall:':<15} {100*test_metrics['recall']:.2f}%")
print(f"{'F1-score:':<15} {100*test_metrics['f1']:.2f}%")
print("\nClass-wise Performance:")
for class_name, metrics in class_metrics.items():
    print(f"\n{class_name}:")
    print(f"  {'Accuracy:':<12} {100*(metrics['true_positives']/metrics['support'] if metrics['support'] > 0 else 0):.2f}%")
    print(f"  {'Precision:':<12} {100*metrics['precision']:.2f}%")
    print(f"  {'Recall:':<12} {100*metrics['recall']:.2f}%")
    print(f"  {'F1:':<12} {100*metrics['f1']:.2f}%")
    print(f"  {'Samples:':<12} {metrics['support']}")
    print("="*60)

# Save model and prediction function
torch.save(model.state_dict(), 'logo_classifier.pth')

def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    image = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        probabilities = F.softmax(output, dim=1)
    return dataset.classes[predicted.item()], probabilities[0].cpu().numpy()


Epoch 1/100
Training:    Loss: 5.6637 | Acc: 66.94%
Validation:  Loss: 0.9089 | Acc: 81.48%
             Precision: 81.41%
             Recall: 81.48%
             F1-score: 81.43%

Epoch 2/100
Training:    Loss: 2.3096 | Acc: 85.95%
Validation:  Loss: 0.7604 | Acc: 82.72%
             Precision: 82.63%
             Recall: 82.72%
             F1-score: 82.62%

Epoch 3/100
Training:    Loss: 1.6555 | Acc: 84.30%
Validation:  Loss: 1.1905 | Acc: 88.89%
             Precision: 90.64%
             Recall: 88.89%
             F1-score: 88.49%

Epoch 4/100
Training:    Loss: 0.7445 | Acc: 92.15%
Validation:  Loss: 1.2964 | Acc: 87.65%
             Precision: 88.20%
             Recall: 87.65%
             F1-score: 87.40%

Epoch 5/100
Training:    Loss: 0.7172 | Acc: 91.74%
Validation:  Loss: 0.6990 | Acc: 87.65%
             Precision: 87.63%
             Recall: 87.65%
             F1-score: 87.59%

Epoch 6/100
Training:    Loss: 0.6747 | Acc: 91.32%
Validation:  Loss: 0.5131 | Acc: 91.3